# Modular mit Ansible Rollen

Würde man seine Infrastruktur mit _Webservern_, _Datenbanken_, _Loadbalancer_, _Queues_ usw. in __einem__ Playbook verwalten, würde dies zu einer riesigen Abfolge von Aktionen ([Spaghetticode](https://de.wikipedia.org/wiki/Spaghetticode)) in __einem__ Skript führen.

Darüber hinaus kann man eine Teil z.B. Datenbanken nicht in anderen Projekten weiter verwenden.

Hier kommen Rollen ins Spiel. Zum Beispiel für:

* nginx
* mysql
* mongodb
* tomcat
* ...

Neben Rollen gibt es noch `include` von Playbooks. Rollen sind aber weit aus mächtiger.

In [1]:
cat ssh-add-passphrase.sh

#!/usr/bin/expect -f
spawn ssh-add /home/vagrant/.ssh/id_rsa
expect "Enter passphrase for /home/vagrant/.ssh/id_rsa:"
send "geheim\n";
interact


In [4]:
chmod u+x ./ssh-add-passphrase.sh

In [5]:
eval `ssh-agent -s` > /dev/null
./ssh-add-passphrase.sh


Enter passphrase for /home/vagrant/.ssh/id_rsa: 
Identity added: /home/vagrant/.ssh/id_rsa (/home/vagrant/.ssh/id_rsa)


In [6]:
ansible -m ping all

127.0.0.1 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
192.168.60.11 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
192.168.60.13 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
192.168.60.12 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
192.168.60.2 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
192.168.60.21 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
192.168.60.22 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


In [7]:
cd Kap2_Rollen

In [8]:
pwd

/home/vagrant/projects/Kap2_Rollen


In [9]:
tree --charset=ascii

.
|-- customhosts
|-- roles
|   |-- base
|   |   `-- tasks
|   |       `-- main.yml
|   `-- nginx
|       |-- files
|       |   |-- default.conf
|       |   `-- index.html
|       |-- handlers
|       |   `-- main.yml
|       |-- meta
|       |   `-- main.yml
|       `-- tasks
|           |-- configure.yml
|           |-- install.yml
|           |-- main.yml
|           `-- service.yml
|-- site.yml
`-- www.yml

8 directories, 12 files


Rollen werden u.a. unter dem Ordner `roles/` abgelegt. Können aber auch über mehrere Ordner verteilt werden, wie z.B. `/deploy/ansible/roles` und `/deploy/ansible/community/roles`.
Dies kann über eine Datei __ansible.cfg__ mit dem Eintrag
    
    roles_path = /deploy/ansible/roles:/deploy/ansible/community/roles
    
geschehen.

Mehr über Rollen hier: http://docs.ansible.com/ansible/playbooks_roles.html

und über ansible.cfg: http://docs.ansible.com/ansible/intro_configuration.html

Jede Rolle bekommt einen Ordner mit ihrem Rolen-Namen. Diese Ordner können weitere Unterordner haben. Der wichtigste Unterordner ist `tasks/`. Darüber hinaus gibt aber noch weitere Ordner wie `handlers/`, `templates/`, `files/`, `meta/`, `vars/` uvw. Normalerweise beinhalten alle diese Ordner eine Datei mit dem Namen `main.yml`.

Wie können Rollen aufgerufen werden? Schauen wir uns das Playbook `site.yml` an:

In [6]:
cat site.yml

---
# This is a sitewide playbook
- include: www.yml


In [10]:
ansible-playbook --syntax-check site.yml


playbook: site.yml


In Playbooks können Teile mit include eingebunden werde. Diese werden im aktuellen Verzeichnis gesucht, wenn kein absoluter/relativer Pfad angegeben wurde.

Mehr über Include: http://docs.ansible.com/ansible/playbooks_roles.html#task-include-files-and-encouraging-reuse

Hier der Inhalt von www.yml

In [11]:
cat www.yml

---
- hosts: www
  remote_user: vagrant
  become: yes
  pre_tasks:
  - debug: 
      msg: 'I":" Beginning to configure web server..'

  roles:
     - nginx

  post_tasks:
  - debug:
      msg: 'I":" Done configuring nginx web server...'


Dieses Playbook wird nur für die Gruppe __www__ ausgeführt.

Der Abschnitt __pre_tasks__ bzw. __post_tasks__ wird vor bzw. nach dem Task Abschnitt ausgeführt.

Unser Task Block wird hier durch eine Rolle __nginx__ abgebildet.

Schauen wir uns zunächst den Meta (Beschreibung) zu dieser Rolle an.

In [14]:
cat roles/nginx/meta/main.yml

---
dependencies:
  - { role: base }


Die Rolle `nginx` ist abhängig von der Rolle `base`. Daher hier die Rolle base.

In [9]:
cat roles/base/tasks/main.yml

---
# essential tasks. should run on all nodes
 - name: creating devops group 
   group: name=devops state=present
 - name: create devops user with admin previleges
   user: name=devops comment="Devops User" uid=2001 group=devops
 - name: install htop package
   action: apt name=htop state=present update_cache=yes


Hier werden unsere User, Gruppen und Grundlegende Pakete installiert.

Schauen wir uns die Rolle `nginx` im Ordner `tasks/` näher an.

In [10]:
cat roles/nginx/tasks/main.yml \
    roles/nginx/tasks/install.yml \
    roles/nginx/tasks/configure.yml \
    roles/nginx/tasks/service.yml

---
# This is main tasks file for nginx role
 - include: install.yml
 - include: configure.yml
 - include: service.yml

 
---
 - name: add official nginx repository
   apt_repository: repo='deb http://nginx.org/packages/ubuntu/ lucid nginx'
 - name: install nginx web server and ensure its at the latest version
   apt: name=nginx state=latest force=yes
---
 - name: create default site configurations 
   copy: src=default.conf dest=/etc/nginx/conf.d/default.conf mode=0644
   notify: 
    - restart nginx service
 - name: create home page for default site
   copy: src=index.html dest=/usr/share/nginx/html/index.html

---
 - name: start nginx service
   service: name=nginx state=started


Die `install.yml` fügt das _ppa nginx repro_ ein. Danach wird die neuste Version von _nginx_ installiert. Auch wenn diese schon installiert wurde (force=yes).

In der `configure.yml` wird die Datei `default.conf` aus dem Ordner `files/` der Rolle nginx auf dem entsprechenden Zielpfad kopiert.

ACHTUNG: Das Modul `copy` schaut im Ordner `files/` nach ob die Datei `default.conf` existiert. Somit sind Rollen unabhängig von ihrer Installation.

In [11]:
cat roles/nginx/files/default.conf

server {
    listen       80;
    server_name  localhost;

    location / {
        root   /usr/share/nginx/html;
        index  index.html;
    }
}


In [12]:
cat roles/nginx/files/index.html

<html>
  <body>
    <h1>Ole Ole Ole </h1>
    <p> Welcome to FIFA World Cup News Portal</p>
   </body>
</html>


In der Datei `configure.yml` wird der Handler "restart nginx service" aufgerufen. Dieser wird über den Ordner `handlers/` in der `main.yml` gesucht und gefunden.

Mehr hierzu unter: http://docs.ansible.com/ansible/playbooks_intro.html#handlers-running-operations-on-change

In [13]:
cat roles/nginx/handlers/main.yml

---
- name: restart nginx service
  service: name=nginx state=restarted



Eine lokales Inventory wird durch die Datei `customhosts` definiert.

In [14]:
cat customhosts

[local]
localhost     ansible_connection=local

[www]
192.168.60.11 ansible_ssh_user=vagrant
192.168.60.12 ansible_ssh_user=vagrant
192.168.60.13 ansible_ssh_user=vagrant

[lb]
192.168.60.2 ansible_ssh_user=vagrant

[db]
192.168.60.21 ansible_ssh_user=vagrant
192.168.60.22 ansible_ssh_user=vagrant


Hier der Aufruf:

In [15]:
ansible-playbook -i customhosts site.yml


PLAY [www] *********************************************************************

TASK [setup] *******************************************************************
ok: [192.168.60.13]
ok: [192.168.60.11]
ok: [192.168.60.12]

TASK [debug] *******************************************************************
ok: [192.168.60.11] => {
    "msg": "I\":\" Beginning to configure web server.."
}
ok: [192.168.60.12] => {
    "msg": "I\":\" Beginning to configure web server.."
}
ok: [192.168.60.13] => {
    "msg": "I\":\" Beginning to configure web server.."
}

TASK [base : creating devops group] ********************************************
changed: [192.168.60.12]
changed: [192.168.60.13]
changed: [192.168.60.11]

TASK [base : create devops user with admin previleges] *************************
changed: [192.168.60.11]
changed: [192.168.60.13]
changed: [192.168.60.12]

TASK [base : install htop package] *********************************************
changed: [192.168.60.11]
changed: [192.168.60.13]

Beachten Sie dabei folgendes:

* Die pre_tasks und post_tasks Ausführung.
* Die Rolle base wird ausgeführt. Warum?

Der Funktionstest könnte so aussehen:

In [16]:
curl 192.168.60.11

<!DOCTYPE HTML><html lang='en' dir='ltr' class='other other0'><head><meta charset="utf-8" /><meta name="referrer" content="no-referrer" /><meta name="robots" content="noindex,nofollow" /><meta http-equiv="X-UA-Compatible" content="IE=Edge"><style id="cfs-style">html{display: none;}</style><link rel="icon" href="favicon.ico" type="image/x-icon" /><link rel="shortcut icon" href="favicon.ico" type="image/x-icon" /><link rel="stylesheet" type="text/css" href="./themes/pmahomme/jquery/jquery-ui-1.11.4.css" /><link rel="stylesheet" type="text/css" href="js/codemirror/lib/codemirror.css?v=4.6.4deb1%2Bdeb.cihar.com%7Exenial.1" /><link rel="stylesheet" type="text/css" href="js/codemirror/addon/hint/show-hint.css?v=4.6.4deb1%2Bdeb.cihar.com%7Exenial.1" /><link rel="stylesheet" type="text/css" href="js/codemirror/addon/lint/lint.css?v=4.6.4deb1%2Bdeb.cihar.com%7Exenial.1" /><link rel="stylesheet" type="text/css" href="phpmyadmin.css.php?nocache=4409427420ltr" /><link rel="stylesheet" type="text/c

OOPS: Da leckt noch Öl raus, da muss man noch bei.

M.a.W: Es wird noch die pma geladen und nicht unsere Webseite. Unsere Rolle muss noch angepasste werden.